In [74]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/modeling-dataset-accident-in-france/modeling_data_accident_in-france.csv


In [75]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier

from imblearn.over_sampling import SMOTE
import pickle

import warnings 
warnings.filterwarnings('ignore')

# For reproducibility
np.random.seed(42)

In [76]:
df = pd.read_csv('../input/modeling-dataset-accident-in-france/modeling_data_accident_in-france.csv')

In [77]:
# We move the target variable to the front, for simplicity
df.insert(0, "accident_severity", df.pop("accident_severity"))
df.head()

,accident_severity,lighting,intersection,atmosphere,collision,localization,user_category,pedestrian_loc,pedestrian_action,pedestrian_company,road_category,traff_regime,reserved_lane,longitud_profile,drawing_plan,surface_cond,infrastructure,hours_sin,hours_cos,acc_situation
0,3,1,1,8.0,3.0,False,1,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0,0.0,-0.345714,-0.938340,1.0
1,3,1,6,1.0,6.0,False,1,0.0,0.0,0.0,3.0,1.0,0.0,1.0,2.0,1.0,0.0,0.005058,0.999987,1.0
2,3,1,1,1.0,6.0,True,3,6.0,2.0,1.0,3.0,2.0,0.0,1.0,3.0,2.0,0.0,0.646868,0.762602,3.0
3,4,2,1,7.0,3.0,False,2,0.0,0.0,0.0,4.0,2.0,0.0,1.0,1.0,1.0,0.0,0.646868,0.762602,1.0
4,3,1,3,1.0,3.0,False,1,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,1.0,0.0,0.985340,0.170603,1.0


In [78]:
df.accident_severity.unique()

array([3, 4, 2])

In [79]:
from sklearn.preprocessing import LabelEncoder

In [80]:
le = LabelEncoder()
df['localization'] = le.fit_transform(df['localization'])

In [81]:
df.head()

,accident_severity,lighting,intersection,atmosphere,collision,localization,user_category,pedestrian_loc,pedestrian_action,pedestrian_company,road_category,traff_regime,reserved_lane,longitud_profile,drawing_plan,surface_cond,infrastructure,hours_sin,hours_cos,acc_situation
0,3,1,1,8.0,3.0,0,1,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0,0.0,-0.345714,-0.938340,1.0
1,3,1,6,1.0,6.0,0,1,0.0,0.0,0.0,3.0,1.0,0.0,1.0,2.0,1.0,0.0,0.005058,0.999987,1.0
2,3,1,1,1.0,6.0,1,3,6.0,2.0,1.0,3.0,2.0,0.0,1.0,3.0,2.0,0.0,0.646868,0.762602,3.0
3,4,2,1,7.0,3.0,0,2,0.0,0.0,0.0,4.0,2.0,0.0,1.0,1.0,1.0,0.0,0.646868,0.762602,1.0
4,3,1,3,1.0,3.0,0,1,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,1.0,0.0,0.985340,0.170603,1.0


In [85]:
dfm = df.copy()

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 839985 entries, 0 to 839984
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   accident_severity   839985 non-null  int64  
 1   lighting            839985 non-null  int64  
 2   intersection        839985 non-null  int64  
 3   atmosphere          839985 non-null  float64
 4   collision           839985 non-null  float64
 5   localization        839985 non-null  int64  
 6   user_category       839985 non-null  int64  
 7   pedestrian_loc      839985 non-null  float64
 8   pedestrian_action   839985 non-null  float64
 9   pedestrian_company  839985 non-null  float64
 10  road_category       839985 non-null  float64
 11  traff_regime        839985 non-null  float64
 12  reserved_lane       839985 non-null  float64
 13  longitud_profile    839985 non-null  float64
 14  drawing_plan        839985 non-null  float64
 15  surface_cond        839985 non-nul

## Scaling and train test split

In [89]:
dfm.head()

,accident_severity,lighting,intersection,atmosphere,collision,localization,user_category,pedestrian_loc,pedestrian_action,pedestrian_company,road_category,traff_regime,reserved_lane,longitud_profile,drawing_plan,surface_cond,infrastructure,hours_sin,hours_cos,acc_situation
0,3,1,1,8.0,3.0,0,1,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0,0.0,-0.345714,-0.938340,1.0
1,3,1,6,1.0,6.0,0,1,0.0,0.0,0.0,3.0,1.0,0.0,1.0,2.0,1.0,0.0,0.005058,0.999987,1.0
2,3,1,1,1.0,6.0,1,3,6.0,2.0,1.0,3.0,2.0,0.0,1.0,3.0,2.0,0.0,0.646868,0.762602,3.0
3,4,2,1,7.0,3.0,0,2,0.0,0.0,0.0,4.0,2.0,0.0,1.0,1.0,1.0,0.0,0.646868,0.762602,1.0
4,3,1,3,1.0,3.0,0,1,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,1.0,0.0,0.985340,0.170603,1.0


In [90]:
dfm.accident_severity.unique()

array([3, 4, 2])

In [91]:
print(sum(dfm.accident_severity == 2))
print(sum(dfm.accident_severity == 3))
print(sum(dfm.accident_severity == 4))

30196
280008
529781


In [92]:
dfm['accident_severity'] = dfm['accident_severity'].map({2:0, 3:1, 4:2 })

In [93]:
print(sum(dfm.accident_severity == 0))
print(sum(dfm.accident_severity == 1))
print(sum(dfm.accident_severity == 2))
dfm.accident_severity.unique()

30196
280008
529781


array([1, 2, 0])

In [94]:
def  split_tables ( ds ): 
    class_dummies  =  pd . get_dummies ( ds [ "accident_severity" ], prefix = "accident_severity" ) 
    ds  =  ds . join ( class_dummies ) 
    return  ds

In [95]:
dfm =  split_tables ( dfm )

In [96]:
dfm

,accident_severity,lighting,intersection,atmosphere,collision,localization,user_category,pedestrian_loc,pedestrian_action,pedestrian_company,...,longitud_profile,drawing_plan,surface_cond,infrastructure,hours_sin,hours_cos,acc_situation,accident_severity_0,accident_severity_1,accident_severity_2
0,1,1,1,8.0,3.0,0,1,0.0,0.0,0.0,...,1.0,3.0,1.0,0.0,-0.345714,-0.938340,1.0,0,1,0
1,1,1,6,1.0,6.0,0,1,0.0,0.0,0.0,...,1.0,2.0,1.0,0.0,0.005058,0.999987,1.0,0,1,0
2,1,1,1,1.0,6.0,1,3,6.0,2.0,1.0,...,1.0,3.0,2.0,0.0,0.646868,0.762602,3.0,0,1,0
3,2,2,1,7.0,3.0,0,2,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.646868,0.762602,1.0,0,0,1
4,1,1,3,1.0,3.0,0,1,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.985340,0.170603,1.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839980,2,5,2,1.0,3.0,0,1,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.985768,0.168111,1.0,0,0,1
839981,2,1,1,1.0,3.0,0,2,0.0,0.0,0.0,...,1.0,2.0,1.0,0.0,0.644938,0.764235,1.0,0,0,1
839982,2,1,2,1.0,3.0,0,2,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,-0.639121,0.769106,1.0,0,0,1
839983,2,1,3,2.0,5.0,0,1,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,-0.868125,-0.496345,1.0,0,0,1


In [97]:
y = dfm['accident_severity']
y_0 = dfm['accident_severity_0']
y_1 = dfm['accident_severity_1']
y_2 = dfm['accident_severity_2']

X = dfm.drop(['accident_severity','accident_severity_0','accident_severity_1','accident_severity_2'], axis=1) 

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y)
X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X,y_0, test_size=0.2, stratify=y)
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X,y_1, test_size=0.2, stratify=y)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X,y_2, test_size=0.2, stratify=y)

In [100]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape )
print(X_train_0.shape, X_test_0.shape, y_train_0.shape, y_test_0.shape)
print(X_train_1.shape, X_test_1.shape, y_train_1.shape, y_test_1.shape)
print(X_train_2.shape, X_test_2.shape, y_train_2.shape, y_test_2.shape)

(671988, 19) (167997, 19) (671988,) (167997,)
(671988, 19) (167997, 19) (671988,) (167997,)
(671988, 19) (167997, 19) (671988,) (167997,)
(671988, 19) (167997, 19) (671988,) (167997,)


### Modeling

In [107]:
def train_pred_report(model, train_data, test_data,train_labels, test_labels):
    
    name = type(model).__name__
    
    #fit the model
    print(f'Fitting {name} model...')
    model.fit(train_data, train_labels)
    
    #make predictions
    print('making predictions...')
    y_pred = model.predict(test_data)
    
    # evaluation metrices
    report = classification_report(test_labels, y_pred)
    
    report_dict = classification_report(test_labels, y_pred, output_dict=True)
    
    return report, report_dict



In [108]:
target_labels = [[y_train,y_test], [y_train_0,y_test_0], [y_train_1,y_test_1], [y_train_2,y_test_2]]


In [109]:
for t in target_labels:
    report, report_dict = train_pred_report(LogisticRegression(penalty='l1',solver='liblinear', n_jobs=-1),X_train, X_test, t[0],t[1])
    print(report)
    print('----------------------------------------------------------------------------------------------------------\n')


Fitting LogisticRegression model...
making predictions...
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      6039
           1       0.57      0.29      0.39     56002
           2       0.69      0.91      0.79    105956

    accuracy                           0.67    167997
   macro avg       0.42      0.40      0.39    167997
weighted avg       0.63      0.67      0.63    167997

----------------------------------------------------------------------------------------------------------

Fitting LogisticRegression model...
making predictions...
              precision    recall  f1-score   support

           0       0.96      1.00      0.98    161958
           1       0.00      0.00      0.00      6039

    accuracy                           0.96    167997
   macro avg       0.48      0.50      0.49    167997
weighted avg       0.93      0.96      0.95    167997

--------------------------------------------------------------------

In [114]:
tree_models = [DecisionTreeClassifier(),
          RandomForestClassifier(n_jobs = -1),
          GradientBoostingClassifier(),
          AdaBoostClassifier(),
          LGBMClassifier()]

# XGBClassifier(n_jobs=-1)

In [ ]:
for m in tree_models:
    for t in target_labels:
        report, report_dict = train_pred_report(m,X_train, X_test, t[0],t[1])
        print(report)
        print('---------------------------------------------------------------------------------------------------------\n')
        
    print('\n=============================================================================================================\n')


Fitting DecisionTreeClassifier model...
making predictions...
making predictions...
              precision    recall  f1-score   support

           0       0.18      0.05      0.07      6039
           1       0.51      0.39      0.44     56002
           2       0.71      0.83      0.77    105956

    accuracy                           0.65    167997
   macro avg       0.47      0.42      0.43    167997
weighted avg       0.62      0.65      0.63    167997

---------------------------------------------------------------------------------------------------------

Fitting RandomForestClassifier model...
making predictions...
              precision    recall  f1-score   support

           0       0.96      1.00      0.98    161958
           1       0.03      0.00      0.01      6039

    accuracy                           0.96    167997
   macro avg       0.50      0.50      0.49    167997
weighted avg       0.93      0.96      0.95    167997

---------------------------------------